<a href="https://colab.research.google.com/github/rich-hyun/Kuggle_9th_team_SAMTARBUCKS/blob/main/Topic_modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##pip & download

In [ ]:
!pip install pyLDAvis #코랩은 install 하고 런타임 다시시작

In [ ]:
!pip install nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

##import

In [ ]:
import nltk
import numpy as np # 넘파이
import pandas as pd # 판다스
import os # (기본 내장 라이브러리) os
import matplotlib.pyplot as plt # 맷플롯립
import seaborn as sns # 씨본
import pandas as pd
from datetime import datetime
import calendar
from gensim import corpora, models
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import re
from collections import Counter
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from nltk.stem import WordNetLemmatizer

from sklearn.decomposition import TruncatedSVD # 특이값 분해
from gensim.models import LdaModel
from gensim.models.coherencemodel import CoherenceModel
import pyLDAvis.gensim
from sklearn.decomposition import NMF
from sklearn.feature_extraction.text import TfidfVectorizer



## 전처리

In [3]:
# 데이터 로딩
df = pd.read_csv('https://raw.githubusercontent.com/rich-hyun/Kuggle_9th_team_/main/reviews_data.csv',  encoding='utf-8')

# 'Date' 열에서 월, 일, 연도를 추출하여 새로운 열에 저장
# 'Reviewed' 다음에 오는 단어를 'mnth' 열에 저장
df['mnth'] = df['Date'].str.extract(r'Reviewed\s+([A-Za-z]+)')
df['day'] = df['Date'].str.extract(r'(\d{1,2}),').astype('int')
df['year'] = df['Date'].str.extract(r'(\d{4})').astype('int')

# 월 이름을 숫자로 매핑하는 딕셔너리 생성
months_map = {
    'Jan': 1, 'Feb': 2, 'March': 3, 'April': 4, 'May': 5, 'June': 6,
    'July': 7, 'Aug': 8, 'Sept': 9, 'Oct': 10, 'Nov': 11, 'Dec': 12
}

# 'mnth' 열의 월 이름을 숫자로 변환
df['mnth_num'] = df['mnth'].map(months_map)

# 월 이름을 숫자로 매핑하는 딕셔너리 생성
month_mapping = {v: k for k,v in enumerate(calendar.month_name)}

# 월 이름을 월 번호로 변환
df['month_num'] = df['mnth'].map(month_mapping)

# 날짜로부터 요일을 계산하여 'day2' 열에 저장
df['day2'] = df.apply(lambda row: datetime.strptime(f"{row['year']}-{row['mnth_num']}-{row['day']}", '%Y-%m-%d').strftime('%A'), axis=1)

# 요일 이름을 숫자로 매핑하는 딕셔너리 생성
day_to_num = {
    'Monday': 0,
    'Tuesday': 1,
    'Wednesday': 2,
    'Thursday': 3,
    'Friday': 4,
    'Saturday': 5,
    'Sunday': 6
}

# 'day2' 열의 요일 이름을 숫자로 변환
df['day2_num'] = df['day2'].map(day_to_num)

# 원하는 열만 선택 (새로운 'day2_num' 열을 포함하도록 업데이트)
df = df[['name', 'location', 'Date', 'year', 'mnth_num', 'day', 'day2_num', 'Rating', 'Review']]
# 'Rating' 또는 'Review'가 NaN인 행을 제거
df.dropna(subset=['Rating', 'Review'], inplace = True)

# 'Review' 컬럼에서 'No Review Text'라는 값을 가진 행을 제거합니다.
df = df.loc[df['Review'] != 'No Review Text']

<ipython-input-3-89c99dffe470>:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(subset=['Rating', 'Review'], inplace = True)


In [5]:
from gensim import corpora, models
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import re
from collections import Counter
from nltk.stem import PorterStemmer

# nltk 불용어
stop_words = set(stopwords.words('english'))
# 불용어 리스트에 상위 4개 단어 추가
#stop_words.update(top_4_words)

def clean(text): # LDA하기 위함
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text) # 특수문자 제거
    text = text.lower() # 소문자 변환
    le=WordNetLemmatizer()
    tokens = word_tokenize(text)
    tokens=[le.lemmatize(w) for w in tokens if w not in stop_words and len(w)>3]
    tokens = [word for word in tokens if word not in stop_words] # 불용어 제거

    return tokens

In [6]:
# vectorizers for creating the document-term-matrix (DTM)
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from nltk.stem import WordNetLemmatizer

#stop-words
stop_words=set(nltk.corpus.stopwords.words('english'))

def clean_text(headline):  # NMF하기 위함
  le=WordNetLemmatizer()
  word_tokens=word_tokenize(headline)
  tokens=[le.lemmatize(w) for w in word_tokens if w not in stop_words and len(w)>3]
  cleaned_text=" ".join(tokens)
  return cleaned_text

df['cleaned review'] = df['Review'].apply(clean_text)

vect =TfidfVectorizer(stop_words=list(stop_words),max_features=1000)
vect_text=vect.fit_transform(df['cleaned review'])

In [7]:
#cleaned review 확인
df[['Review','cleaned review']].head(4)

,Review,cleaned review
0,Amber and LaDonna at the Starbucks on Southwes...,Amber LaDonna Starbucks Southwest Parkway alwa...
1,** at the Starbucks by the fire station on 436...,Starbucks fire station Altamonte Springs made ...
2,I just wanted to go out of my way to recognize...,wanted recognize Starbucks employee Billy Fran...
3,Me and my friend were at Starbucks and my card...,friend Starbucks card work Thankful worker pai...


##LSA

In [9]:
from sklearn.decomposition import TruncatedSVD # 특이값 분해
lsa_model = TruncatedSVD(n_components=4, algorithm='randomized', n_iter=10, random_state=42)

lsa_top=lsa_model.fit_transform(vect_text)

In [10]:
# most important words for each topic
vocab = vect.get_feature_names_out()

for i, comp in enumerate(lsa_model.components_):
    vocab_comp = zip(vocab, comp)
    sorted_words = sorted(vocab_comp, key= lambda x:x[1], reverse=True)[:5]
    print("Topic "+str(i)+": ")
    for t in sorted_words:
        print(t[0],end=" ")
    print("\n")

Topic 0: 
starbucks coffee drink customer time 

Topic 1: 
card gift account balance starbucks 

Topic 2: 
coffee card starbucks gift taste 

Topic 3: 
customer service great always starbucks 



##LDA

In [11]:
df['cleaned review'] = df['Review'].apply(clean)
df['tokens'] = df['Review'].apply(clean)

# 사전 생성
dictionary = corpora.Dictionary(df['tokens'] )

# 빈도가 낮은 단어 제거
dictionary.filter_extremes(no_below=30, no_above=0.5)

# 단어 빈도 계산
corpus = [dictionary.doc2bow(text) for text in df['tokens'] ]

In [12]:
from gensim.models import LdaModel
from gensim.models.coherencemodel import CoherenceModel

# 토픽의 개수 범위를 정의합니다.
topics_range = range(2, 9, 1)

# 토픽의 개수에 따른 일관성 점수를 저장할 리스트를 생성합니다.
coherence_values = []

# 각 토픽의 개수에 대해 LDA 모델을 학습하고, 일관성 점수를 계산합니다.
for num_topics in topics_range:
    model = LdaModel(corpus=corpus,
                     id2word=dictionary,
                     num_topics=num_topics,
                     random_state=100,
                     update_every=1,
                     chunksize=1000,
                     passes=15,
                     alpha='auto',
                     per_word_topics=True)

    coherencemodel = CoherenceModel(model=model, texts=corpus, dictionary=dictionary, coherence='c_v')
    coherence_values.append(coherencemodel.get_coherence())

# 일관성 점수가 가장 높은 토픽의 개수를 구합니다.
optimal_num_topics = topics_range[coherence_values.index(max(coherence_values))]

print("Optimal number of topics: ", optimal_num_topics)


/usr/local/lib/python3.10/dist-packages/gensim/topic_coherence/direct_confirmation_measure.py:204: RuntimeWarning: divide by zero encountered in scalar divide
  m_lr_i = np.log(numerator / denominator)
/usr/local/lib/python3.10/dist-packages/gensim/topic_coherence/indirect_confirmation_measure.py:323: RuntimeWarning: invalid value encountered in scalar divide
  return cv1.T.dot(cv2)[0, 0] / (_magnitude(cv1) * _magnitude(cv2))


Optimal number of topics:  2


In [13]:
# LDA 모델 생성 .. coherence는 토픽의 일관
lda_model = models.LdaModel(corpus=corpus,
                     id2word=dictionary,
                     num_topics = 3,
                     random_state=100,
                     update_every=1,
                     chunksize=1000,
                     passes=20,
                     alpha='auto',
                     per_word_topics=True)
# 토픽 출력
topics = lda_model.print_topics(num_words = 4)
for topic in topics:
    print(topic)

(0, '0.097*"card" + 0.033*"customer" + 0.029*"told" + 0.026*"time"')
(1, '0.062*"coffee" + 0.050*"drink" + 0.032*"said" + 0.027*"order"')
(2, '0.059*"customer" + 0.053*"coffee" + 0.042*"service" + 0.039*"store"')


In [14]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary)
pyLDAvis.display(vis)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


##NMF

In [15]:
from sklearn.decomposition import NMF
from sklearn.feature_extraction.text import TfidfVectorizer



# NMF 모델 생성 및 학습
n_topics = 4  # 토픽의 개수를 설정합니다.
nmf = NMF(n_components=n_topics, random_state=0)
nmf.fit(vect_text)

# 토픽과 키워드 출력
feature_names = vect.get_feature_names_out()
for topic_idx, topic in enumerate(nmf.components_):
    print(f"Topic #{topic_idx + 1}")
    print(" ".join([feature_names[i] for i in topic.argsort()[:-10 - 1:-1]]))

# 각 문서의 토픽 예측
topic_values = nmf.transform(vect_text)

# 가장 확률이 높은 토픽을 선택
topics = np.argmax(topic_values, axis=1)

# 예측된 토픽을 새로운 열로 추가
df['Topic'] = topics

Topic #1
customer service starbucks store always employee location time year great
Topic #2
card starbucks gift account balance told email called money credit
Topic #3
coffee starbucks like good taste food place shop cream tasted
Topic #4
drink said order ordered went asked milk time back could


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [16]:
df['Topic'].value_counts()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Topic
3    227
0    212
2    176
1     84
Name: count, dtype: int64

In [17]:
df[['Review','Topic']]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Review,Topic
0,Amber and LaDonna at the Starbucks on Southwes...,0
1,** at the Starbucks by the fire station on 436...,3
2,I just wanted to go out of my way to recognize...,0
3,Me and my friend were at Starbucks and my card...,1
4,I?셫 on this kick of drinking 5 cups of warm wa...,2
...,...,...
696,I ordered Via Starbucks coffee online. I recei...,1
697,"My name is Ric **, I am journalist by professi...",2
698,"The bagel was ice cold, not cut and not toasted.",3
699,"In the morning of Monday, August 15, 2011, at ...",3


- 한 리뷰에 여러가지 토픽이 중복돼있음

#BERTOPIC

In [23]:
!pip install bertopic

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


  Using cached bertopic-0.15.0-py2.py3-none-any.whl (143 kB)
  Using cached hdbscan-0.8.33.tar.gz (5.2 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached umap-learn-0.5.4.tar.gz (90 kB)
  Preparing metadata (setup.py) ... done
  Using cached sentence-transformers-2.2.2.tar.gz (85 kB)
  Preparing metadata (setup.py) ... done
  Using cached Cython-0.29.36-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.manylinux_2_24_x86_64.whl (1.9 MB)
  Using cached transformers-4.35.0-py3-none-any.whl (7.9 MB)
  Using cached sentencepiece-0.1.99-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.3 MB)
  Using cached huggingface_hub-0.19.1-py3-none-any.whl (311 kB)
  Using cached pynndescent-0.5.10.tar.gz (1.1 MB)
  Preparing metadata (setup.py) ... done
  Using cached tokenizers-0.14.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.8 MB)
  Using cached safetensors-0.4

In [24]:
from bertopic import BERTopic
model = BERTopic(nr_topics="auto")
# vect_text 안의 모든 항목을 문자열로 변환
#vect_text = [str(text) for text in df['cleaned review'] ]

# 변환된 vect_text를 다시 BERTopic 모델에 적용
topics, probabilities= model.fit_transform(df['cleaned review'] )

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/skimage/util/dtype.py:27: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  np.bool8: (False, True),


/usr/local/lib/python3.10/dist-packages/hdbscan/hdbscan_.py:1170: DeprecationWarning: `alltrue` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `all` instead.
  self._all_finite = is_finite(X)
/usr/local/lib/python3.10/dist-packages/hdbscan/hdbscan_.py:1170: DeprecationWarning: `alltrue` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `all` instead.
  self._all_finite = is_finite(X)


In [25]:
print('각 문서의 토픽 번호 리스트 :',len(topics))
print('첫번째 문서의 토픽 번호 :', topics[0])

각 문서의 토픽 번호 리스트 : 699
첫번째 문서의 토픽 번호 : 1


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [26]:
model.get_topic_info()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Topic,Count,Name,Representation,Representative_Docs
0,-1,251,-1_starbucks_coffee_store_customer,"[starbucks, coffee, store, customer, time, lik...",[would like tell absolutely humiliating experi...
1,0,201,0_starbucks_customer_order_coffee,"[starbucks, customer, order, coffee, said, dri...",[When ordering morning clearly asked Venti dri...
2,1,92,1_coffee_starbucks_always_good,"[coffee, starbucks, always, good, great, servi...",[Starbucks daily coffee good great order small...
3,2,56,2_card_starbucks_account_gift,"[card, starbucks, account, gift, balance, call...",[trusted Starbucks stored three Starbucks card...
4,3,27,3_drink_coffee_charge_starbucks,"[drink, coffee, charge, starbucks, charged, ve...",[last couple week start seeing quad espresso g...
5,4,26,4_drink_ordered_milk_iced,"[drink, ordered, milk, iced, caramel, stomach,...",[typically order thing iced caramel macchiato ...
6,5,18,5_coffee_starbucks_christmas_drink,"[coffee, starbucks, christmas, drink, flavor, ...",[Starbucks getting pricey whatever drink Every...
7,6,17,6_reward_star_program_free,"[reward, star, program, free, birthday, starbu...",[When found today Starbucks reward program cou...
8,7,11,7_sandwich_cheese_bacon_muffin,"[sandwich, cheese, bacon, muffin, cooked, gave...",[December 2014 12:49:35 went usual Starbucks u...


In [27]:
model.get_topic(5)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[('coffee', 0.07204857408635341),
 ('starbucks', 0.0686319375662379),
 ('christmas', 0.04804898824966219),
 ('drink', 0.04708438931628265),
 ('flavor', 0.04258060439131107),
 ('year', 0.04184348877523334),
 ('price', 0.040784347450920064),
 ('milkshake', 0.037751460033942225),
 ('stainless', 0.036488999311717236),
 ('steel', 0.035457824100214794)]

In [28]:
model.visualize_topics()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [29]:
model.visualize_heatmap()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.

